# Good Film // Bad Film
## Regression for Box Office Revenue and IMdB Critic Scores

This notebook builds and evaluates a handful of regression models which predict critical reception scores for films. Independent variables include plot synopsis free text, social media metrics on the leading actors, and other categorical variables such as film genre.

In [2]:
# Data Wrangling
import pandas as pd
import numpy as np
from scipy.stats import boxcox

# Viz
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
sns.set(style="white")

# Python Extras
import os
import time
import pickle
from unicodedata import normalize

# API Calls and Parsing
import requests
from pycountry import languages

# NLP Tools
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Model Building and Prep
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score

***

# Data Collection

## Kaggle Dataset

From https://www.kaggle.com/tmdb/tmdb-movie-metadata:

In [3]:
kaggle_data = pd.read_csv('../data/imdb_5000_movies.csv') # Just a big Kaggle dataset full of movies.

In [4]:
kaggle_data.shape

(5043, 28)

## Extra Metadata from TheMovieDB.org

One good thing about this dataset is that it provides the unique IMdB movie ID's, which we can pass to a third-party API in order to supplement our data with more features. Here we parse out those IMdB ID's:

In [5]:
kaggle_data.tail()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
5038,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,Comedy|Drama,...,6.0,English,Canada,NaN,NaN,2013.0,470.0,7.7,NaN,84
5039,Color,NaN,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,Crime|Drama|Mystery|Thriller,...,359.0,English,USA,TV-14,NaN,NaN,593.0,7.5,16.00,32000
5040,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,Drama|Horror|Thriller,...,3.0,English,USA,NaN,1400.0,2013.0,0.0,6.3,NaN,16
5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660
5042,Color,Jon Gunn,43.0,90.0,16.0,16.0,Brian Herzlinger,86.0,85222.0,Documentary,...,84.0,English,USA,PG,1100.0,2004.0,23.0,6.6,1.85,456


Grab IMdB ID numbers for each film and append to dataframe:

In [6]:
imdb_ids = kaggle_data['movie_imdb_link'].str[26:35]
# imdb_ids[3000:3005]

In [7]:
kaggle_data['imdb_ids'] = imdb_ids

Here we make the API calls, and persist our data to little .json files:

In [8]:
try:
    os.mkdir('../data/movie_metadata')
except FileExistsError:
    pass

__This will make lots of API calls, be warned and use sparingly__, to avoid burning out your API key

You will need to enter your own API key in the API_KEY.py file (remove the .template suffix)

In [9]:
# from API_KEY import API_KEY
# i=0; j=0; total=len(imdb_ids)

# for id in imdb_ids:
#     print(f"Downloading movie {i} of {total}...")
#     i+=1
#     query_string = f'https://api.themoviedb.org/3/movie/{id}?api_key={API_KEY}'
#     json = requests.get(query_string).text
#     if "could not be found" in json:
#         j+=1
#         print(f"{round(j/i, 2)}% of movies not found")
#         continue
#     f = open(f'../data/movie_metadata/movie_{id}.json', 'w+')
#     f.write(json)
#     f.close()

## Combine Kaggle Dataset and TheMovieDB API Responses

In [10]:
tmdb_movies = pd.DataFrame()

for filename in os.listdir(path='../data/movie_metadata/'):
    if "movie" in filename:
        this_movie = pd.read_json(f"../data/movie_metadata/{filename}", lines=True)
        tmdb_movies = pd.concat([this_movie,tmdb_movies], axis=0)

tmdb_movies = tmdb_movies.reset_index(drop=True)

KeyboardInterrupt: 

In [ ]:
df_merged = tmdb_movies.merge(kaggle_data, 
                              how='left',
                              left_on='imdb_id',
                              right_on='imdb_ids',
                              suffixes=("_kaggle","_api"))

In [ ]:
df_merged.shape

***

# Data Cleaning and EDA

## Keep only interesting columns

In [ ]:
df_merged.columns

In [ ]:
desired_cols = ("imdb_id duration adult budget_api budget_kaggle language original_language "
                "production_countries runtime spoken_languages color genres_api "
                "title movie_title original_title overview content_rating "
                "actor_1_facebook_likes actor_2_facebook_likes actor_3_facebook_likes "
                "director_facebook_likes cast_total_facebook_likes facenumber_in_poster "
                "aspect_ratio imdb_score revenue").split()

In [ ]:
df_lesscols = df_merged[desired_cols]
df_lesscols.head()

## Drop Duplicate Rows

In [ ]:
df_lesscols[
    df_lesscols.imdb_id.duplicated(keep=False)
].sort_values('imdb_id')[
    "imdb_id duration title language".split()
].head(51)

In [ ]:
df_lesscols[
    df_lesscols.imdb_id.duplicated(keep=False)
].sort_values('imdb_id')[
    "imdb_id duration title language".split()
].tail(50)

In [ ]:
df_lesscols = df_lesscols.drop_duplicates(subset='imdb_id').set_index("imdb_id")

In [ ]:
df_lesscols.head(3)

## Null Values Management

Here are all of the truly "null" values:

In [ ]:
nulls = df_lesscols.isna().sum()[df_lesscols.isna().sum() > 0].sort_values(ascending=False)

nulls.plot(kind='bar', figsize=(15,8)).plot()

Gotta start somewhere.

### Budget

We have two disparate budget columns to draw from. We want whatever seems most plausible.

In [ ]:
df_lesscols[["budget_api", "budget_kaggle"]].head(10)

Lots of nulls in budget_api, lots of zeroes in budget_kaggle. How best to handle this? And are there other columns where we just have a bunch of zero values instead of nulls (revenue??)

In [ ]:
def best_guess_budget(row):
    # When budget_api is NaN and budget_kaggle is non-zero, take budget_kaggle.
    if row.isna()["budget_api"] and row['budget_kaggle'] != 0:
        return row['budget_kaggle']
    
    # When budget_api is not NaN but budget_kaggle is zero, take budget_api.
    elif not row.isna()["budget_api"] and row['budget_kaggle'] == 0:
        return row['budget_api']
    
    # When both values are not NaN / non-zero, take the mean?
    elif not row.isna()["budget_api"] and row['budget_kaggle'] != 0:
        return np.mean((row['budget_kaggle'], row['budget_api']))
        
    # When budget_api is NaN AND budget_kaggle is zero...that's tough. Maybe drop row. Consider imputing values?
    else:
        return None

In [ ]:
df_lesscols.head(10).apply(best_guess_budget, axis=1)

This is the desired behavior.

In [ ]:
best_budget = df_lesscols.apply(best_guess_budget, axis=1)

In [ ]:
df_best_guess_budget = df_lesscols.assign(best_budget=best_budget
                                         ).drop("budget_api budget_kaggle".split(), axis=1)

In [ ]:
df_best_guess_budget.best_budget.isna().mean()

5% of budget values are missing. I think I will in fact impute the median.

In [ ]:
budget_med = df_best_guess_budget.best_budget.dropna().median()
budget_mean = df_best_guess_budget.best_budget.dropna().mean()
budget_med, budget_mean

In [ ]:
df_filled_budget = df_best_guess_budget.assign(
    filled_budget = df_best_guess_budget["best_budget"].fillna(budget_med)
).drop(["best_budget"],axis=1)

In [ ]:
df_filled_budget.filled_budget.isna().mean()

In [ ]:
df_filled_budget.filled_budget.median(), df_filled_budget.filled_budget.mean()

This doesn't seem to have shifted the mean much, that's good.

### Aspect Ratio

In [ ]:
df_filled_budget.aspect_ratio.isna().mean()

6% of aspect ratios are null.

This is a categorical describing the dimensions of the projected image. Your TV set is 16x9 aka 1.77 aspect ratio, whereas lots of hollywood films are 1.85 ratio or 2.40, much "wider-screen". 2.40 might be "artsier".

In [ ]:
# Aspect ratio of 16 I take to mean 16x9 aka 1.77:
df_filled_budget.aspect_ratio = df_filled_budget.aspect_ratio.apply(lambda x: 1.77 if x==16 else x)
df_filled_budget.aspect_ratio.value_counts().sort_index().plot(kind="bar")

I think I will lump some of these together and impute 1.85 on null values. It's not *quite* the mode but it's a sort of middle-of-the road aspect ratio. The DCI standard lists 2.39 and 1.85 for theatrical projection. 16x9 or 1.77 is also common, and 4/3 or 1.33 is like an old-timey boxey aspect ratio.

In [ ]:
aspect_bins = [0, np.mean((1.33, 1.77)), np.mean((1.77,1.85)), np.mean((1.85,2.39)), np.inf]
aspect_labels = "1.33 1.77 1.85 2.39".split()

In [ ]:
pd.cut(df_filled_budget.aspect_ratio, 
       bins=aspect_bins, 
       labels=aspect_labels,
       include_lowest=True)

Lump values and impute 1.85:

In [ ]:
df_fixed_aspect = df_filled_budget.assign(fixed_aspect = pd.cut(df_filled_budget.aspect_ratio, 
                                                         bins=aspect_bins, 
                                                         labels=aspect_labels,
                                                         include_lowest=True
                                                               ).fillna("1.85")
                                         ).drop(["aspect_ratio"],axis=1)

In [ ]:
df_fixed_aspect.fixed_aspect.value_counts().sort_index().plot(kind="bar")

### Content Rating

In [ ]:
df_fixed_aspect.content_rating.isna().mean()

In [ ]:
df_fixed_aspect.content_rating.value_counts().plot(kind="bar")

Just gonna fill nulls with PG-13 and condense redundant ones

In [ ]:
df_content_rating_filled = df_fixed_aspect.assign(
                                content_rating=df_fixed_aspect.content_rating.fillna("PG-13")
                           )

In [ ]:
def rating_mapper(rating):
    if rating == "Not Rated":
        return "Unrated"
    elif rating in "Approved Passed M TV-14".split():
        return "PG-13"
    elif rating in "TV-G GP G TV-PG".split():
        return "PG"
    elif rating in ["NC-17"]:
        return "X"
    else:
        return rating

In [ ]:
df_content_rating_condensed = df_content_rating_filled.assign(
                                content_rating = df_content_rating_filled.content_rating.map(rating_mapper)
                              )

In [ ]:
df_content_rating_condensed.content_rating.value_counts().plot(kind="bar")

In [ ]:
df_content_rating_condensed.content_rating.isna().mean()

How are we doing on null values globally?

In [ ]:
nulls_update = df_content_rating_condensed.isna().sum()[
    df_content_rating_condensed.isna().sum() > 0
].sort_values(ascending=False)

nulls_update.plot(kind='bar', figsize=(15,8), rot=45).plot()

Clearing some memory:

In [ ]:
del nulls
del imdb_ids
del kaggle_data
del best_budget
del df_lesscols
del df_best_guess_budget
del df_filled_budget
del df_content_rating_filled
del df_fixed_aspect

### Facebook Like Counts

In [ ]:
facebook_like_cols = [col for col in df_content_rating_condensed if "facebook" in col]
likes = df_content_rating_condensed[facebook_like_cols]

df_content_rating_condensed.drop(facebook_like_cols, axis=1, inplace=True)

likes.head()

In [ ]:
(round(likes.isna().mean()*100,2)).sort_values(ascending=False).astype('str')+"%"

Not that many values are missing. Will just use the medians.

In [ ]:
likes.fillna({col:likes[col].median() for col in likes.columns}, inplace=True)

In [ ]:
df_likes_filled = pd.concat([df_content_rating_condensed,
                             likes], axis=1)

In [ ]:
del df_content_rating_condensed
del likes

Time for another check-in:

In [ ]:
nulls_update = df_likes_filled.isna().sum()[
    df_likes_filled.isna().sum() > 0
].sort_values(ascending=False)

nulls_update.plot(kind='bar', figsize=(15,8), rot=45).plot()

del nulls_update

### Color

In [ ]:
df_likes_filled.color = df_likes_filled.color.str.lstrip()
df_likes_filled.color.value_counts()

Seems safe to assume that the last dozen films are in color...

In [ ]:
df_likes_filled.color = df_likes_filled.color.fillna(df_likes_filled.color.mode()[0])
df_color_filled = df_likes_filled.copy()

In [ ]:
del df_likes_filled

In [ ]:
df_color_filled.head(3)

In [ ]:
df_color_filled.color.isna().mean()

### Face Number in Poster

In [ ]:
df_color_filled.facenumber_in_poster.value_counts().plot(kind='bar')

Zero seems to be a placeholder for "idk" in this dataset, I'll drop this column.

In [ ]:
df_no_facenums = df_color_filled.drop(['facenumber_in_poster'],axis=1)
del df_color_filled
df_no_facenums.head()

### Language ("original... spoken... language")

In [ ]:
langs = df_no_facenums[[col for col in df_no_facenums.columns if "language" in col]]
langs.head(15)

In [ ]:
langs.isna().mean()

In [ ]:
df_no_facenums.iloc[13,:]

The "language" column looks not great. "original language" is probably a better bet.

In [ ]:
df_no_facenums.spoken_languages.tail()

In [ ]:
df_no_facenums.original_language.tail()

Yeah I think that's probably the best column.

In [ ]:
df_no_facenums.original_language.value_counts().head(10)

Using a python package to make language codes human-readable:

In [ ]:
lang_decoder = {lang.alpha_2: lang.name for lang in languages if hasattr(lang,'alpha_2')}
list(lang_decoder.items())[:5]

In [ ]:
df_no_facenums.original_language = df_no_facenums.original_language.map(lang_decoder)
df_no_facenums.original_language.value_counts()[2:].plot(kind='bar')

Just eyeballing it...I think anything after German, in terms of frequency, is gonna have to go in "other"

Grabbing most popular languages:

In [ ]:
top_langs = df_no_facenums.original_language.value_counts().index[:3].tolist()
top_langs

In [ ]:
def language_lumper(lang):
    if lang not in top_langs:
        return "Other"
    else:
        return lang

In [ ]:
df_no_facenums.original_language = df_no_facenums.original_language.apply(language_lumper)

In [ ]:
df_langs_lumped = df_no_facenums.drop("language spoken_languages".split(),axis=1)
del df_no_facenums
del langs

In [ ]:
df_langs_lumped.head()

In [ ]:
df_langs_lumped.original_language.value_counts().plot(kind='bar')

***

In [ ]:
nulls = df_langs_lumped.isna().sum()[df_langs_lumped.isna().sum() > 0].sort_values(ascending=False)

nulls.plot(kind='bar', figsize=(15,8)).plot()

del nulls

### Duration / Runtime

In [ ]:
durs = df_langs_lumped["duration runtime".split()]
durs.head()

How different are these columns?

In [ ]:
(durs.duration - durs.runtime).plot(kind='hist')
(durs.duration - durs.runtime).mean()

Not very.

In [ ]:
df_langs_lumped.drop(["duration"],axis=1,inplace=True)

In [ ]:
df_runtimes_filled = df_langs_lumped.assign(
    durations=df_langs_lumped.runtime.fillna(
        df_langs_lumped.runtime.median()
    )
).drop(["runtime"],axis=1)

del df_langs_lumped
del durs

In [ ]:
df_runtimes_filled.isna().mean()

**No more obvious nulls!**

In [ ]:
df_runtimes_filled.to_pickle("../data/pickles/df_no_nulls.pkl")

## Categorical Columns

In [ ]:
df_no_nulls = pd.read_pickle("../data/pickles/df_no_nulls.pkl")
df_no_nulls.columns.tolist()

### "Adult"

In [ ]:
df_no_nulls.adult.value_counts()

Single-value column. Drop it

In [ ]:
df_no_adult = df_no_nulls.drop(["adult"],axis=1)
del df_no_nulls

In [ ]:
df_no_adult.columns

### Production Countries

In [ ]:
df_no_adult['production_countries'].head()

Hmm list of dict. Let's just grab the human-readable names:

In [ ]:
def prod_countries_extractor(countries):
    if len(countries)!=0:
        return {country['name'] for country in countries}
    else:
        return {}

In [ ]:
df_no_adult.production_countries = df_no_adult.production_countries.apply(prod_countries_extractor)

In [ ]:
df_no_adult.production_countries.iloc[29:34]

In [ ]:
# Turns out pandas vectorized string operations work on lists too
ax = df_no_adult.production_countries.str.len().value_counts().sort_index().plot(kind='bar')
ax.set_xlabel("N countries")
ax.set_ylabel("N Films")
ax.plot()
del ax

I feel like that ^ can become an int column, after the zeroes are fixed

In [ ]:
# If it's not in English, can we guess where it was filmed?
df_no_adult[(
                df_no_adult.production_countries.str.len()==0
            )&(
                df_no_adult.original_language!="English"
            )]

In [ ]:
guesses = {480:"Germany", 
           2051:"Germany",
           2260:"India",
           3242:"India"}

for index, country in guesses.items():
    df_no_adult['production_countries'].iloc[index] = country
    
del index
del country

In [ ]:
pd.DataFrame([df_no_adult.iloc[i,:] for i in guesses.keys()])

In [ ]:
del guesses

In [ ]:
df_no_adult.production_countries.value_counts().head(10)

In [ ]:
def fill_with_USA(country_set):
    return {"United States of America"} if country_set == {} else country_set

In [ ]:
df_no_adult["production_countries"] = df_no_adult.production_countries.apply(fill_with_USA)

In [ ]:
df_filled_USA = df_no_adult.copy()
del df_no_adult

Might need to split into USA and not-USA to deal with class imbalance

In [ ]:
def usa_or_not(country_set):
    return 0 if country_set=={"United States of America"} else 1

In [ ]:
ax = df_filled_USA.production_countries.apply(usa_or_not).value_counts().plot(kind='bar')
ax.set_xlabel("Shot only in USA")
del ax

In [ ]:
df_filled_USA['shot_only_in_USA'] = df_filled_USA.production_countries.apply(usa_or_not)

In [ ]:
def shot_in_usa_and_abroad(country_set):
    if len(country_set)>1 and "United States of America" in country_set:
        return 1
    else:
        return 0

In [ ]:
df_filled_USA['shot_in_USA_and_abroad'] = df_filled_USA.production_countries.apply(shot_in_usa_and_abroad)

In [ ]:
df_filled_USA['shot_in_USA_and_abroad'].value_counts().plot(kind='bar')

In [ ]:
df_filled_USA['n_production_countries'] = df_filled_USA.production_countries.str.len()
df_filled_USA['n_production_countries'].plot(kind='box')

In [ ]:
df_n_countries = df_filled_USA.drop(["production_countries"],axis=1)

In [ ]:
del df_filled_USA

In [ ]:
df_n_countries.columns.tolist()

### Genre

In [ ]:
df_n_countries['genre'] = df_n_countries['genres_api'].str.split("|")
df_n_countries['genre'] = df_n_countries['genre'].apply(lambda x: set(x))
del df_n_countries["genres_api"]

In [ ]:
df_n_countries['genre'].apply(lambda x: set(x)).value_counts().head(20)

In [ ]:
genre_cats = "Drama Comedy Romance Crime Thriller Horror Action Mystery Sci-Fi Adventure Documentary".split()

In [ ]:
def genre_encoder(genre_set):
    new = pd.Series(0, index=genre_cats)
    for genre in genre_set:
        if genre in genre_cats:
            new[genre] = 1
    return new

In [ ]:
df_n_countries.genre.head()

In [ ]:
df_n_countries.genre.head().apply(genre_encoder)

In [ ]:
genre_matrix = df_n_countries.genre.apply(genre_encoder)

In [ ]:
del df_n_countries['genre']

In [ ]:
genre_matrix.columns = ["".join(["genre_", col]) for col in genre_matrix.columns.tolist()]

In [ ]:
df_encoded_genres = pd.concat([df_n_countries,genre_matrix],axis=1)

In [ ]:
del df_n_countries
del genre_matrix
del genre_cats

In [ ]:
df_encoded_genres.to_pickle("../data/pickles/df_encoded_genres.pkl")

## Numerical Columns

In [ ]:
df_encoded_genres = pd.read_pickle("../data/pickles/df_encoded_genres.pkl")

In [ ]:
df_encoded_genres.columns

### Facebook Likes

In [ ]:
likes = df_encoded_genres[[col for col in df_encoded_genres.columns if "likes" in col]]
likes.head()

In [ ]:
likes.actor_1_facebook_likes.describe()

In [ ]:
likes.actor_1_facebook_likes.hist()

Could probably use a log transform or something. Or is it just full of zeroes?

In [ ]:
likes.actor_1_facebook_likes[likes.actor_1_facebook_likes < 5].shape

In [ ]:
likes.director_facebook_likes[likes.director_facebook_likes == 0].shape

In [ ]:
likes.director_facebook_likes.describe()

I'm willing to believe that directors are just generally not as popular as their actors

In [ ]:
# +1 like for boxcox, does not like zeroes
pd.Series(boxcox(likes.actor_1_facebook_likes+1)[0]).hist()

Seems good

In [ ]:
all_boxcoxed = [pd.Series(boxcox(likes[col]+1)[0], name=f"{col}_box") for col in likes.columns]

In [ ]:
boxed_df = pd.concat(all_boxcoxed,axis=1)
boxed_df.index = df_encoded_genres.index
boxed_df.head()

In [ ]:
for col in boxed_df:
    plt.figure()
    plt.hist(boxed_df[col])
    print(col)
    plt.show()
del col

In [ ]:
no_likes = df_encoded_genres.drop([col for col in df_encoded_genres if "likes" in col], axis=1)

In [ ]:
boxed_df.index = no_likes.index

In [ ]:
df_boxcoxed_likes = pd.concat([no_likes, boxed_df],axis=1)

In [ ]:
df_boxcoxed_likes.head(3)

### Budget

In [ ]:
df_boxcoxed_likes.filled_budget.hist()

Also pretty skewed up

In [ ]:
df_boxcoxed_likes.filled_budget.transform(np.log).hist()

Not perfect, boxcox again?

In [ ]:
budget_boxed = pd.Series(boxcox(df_boxcoxed_likes.filled_budget)[0], name="budget_boxed")
budget_boxed.hist()

In [ ]:
df_boxcoxed_budget = df_boxcoxed_likes.assign(budget_boxed=budget_boxed.values).drop(['filled_budget'],axis=1)

In [ ]:
df_boxcoxed_budget.head()

### Duration

In [ ]:
df_boxcoxed_budget.durations.hist(bins=50)

Seems pretty tightly clustered around feature length film length

In [ ]:
df_boxcoxed_budget.durations.plot(kind='box',figsize=(2,6))

In [ ]:
pd.Series(boxcox(df_boxcoxed_budget.durations+1)[0], name=f"duration_box").hist(bins=50)

I think we can just stick with the original durations, no transform needed.

### Number of Production Countries

In [ ]:
df_boxcoxed_budget.n_production_countries.value_counts().plot(kind='bar')

I should lump together 5+ perhaps...it gets a bit sparse

In [ ]:
df_boxcoxed_budget.n_production_countries.apply(lambda x: 5 if x >= 5 else x).value_counts().plot(kind='bar')

In [ ]:
df_boxcoxed_budget["n_prod_locs_trunc"] = df_boxcoxed_budget.n_production_countries.apply(
                                                                                    lambda x: 5 if x >= 5 else x
                                                                                )

We will need to remember that here, the int 5 represents 5 or more countries of filming.

In [ ]:
del df_boxcoxed_budget["n_production_countries"]

### Revenue (Secondary Response Variable)

My understanding is that response variables do not need to be normally distributed, for regression.

In [ ]:
df_boxcoxed_budget.revenue.hist(bins=50)

In [ ]:
(df_boxcoxed_budget["revenue"]+1).transform(np.log).hist(bins=50)

Hrmm

### IMdB Score (Response Variable)

In [ ]:
df_boxcoxed_budget.imdb_score.describe()

In [ ]:
df_boxcoxed_budget.imdb_score.hist()

In [ ]:
df_boxcoxed_budget.to_pickle("../data/pickles/df_boxcoxed_budget.pkl")

## Free Text Columns

In [ ]:
df_boxcoxed_budget = pd.read_pickle("../data/pickles/df_boxcoxed_budget.pkl")

### Titles

In [ ]:
titles = df_boxcoxed_budget["title movie_title original_title".split()]
titles.head(50)

In [ ]:
titles['title'].iloc[0] == titles["movie_title"].iloc[0]

In [ ]:
titles['title'].iloc[0]

In [ ]:
titles["movie_title"].iloc[0]

In [ ]:
titles['movie_title'] = titles['movie_title'].apply(lambda title: normalize('NFKD', title).strip())

In [ ]:
titles['title'].iloc[0] == titles["movie_title"].iloc[0]

In [ ]:
titles[titles.title != titles.movie_title].shape

In [ ]:
titles[titles.title != titles.movie_title].head(30)

I can't tell just from looking which of these is the definitive "right" column to use. Original_title seems more likely to be in a foreign language...

In [ ]:
titles[titles.title != titles.movie_title].tail(30)

I don't think there is going to be predictive information in the title itself. Maybe I'll just grab one as a plaintext reference to the datapoint...

In [ ]:
df_boxcoxed_budget.drop(['movie_title', "original_title"],axis=1,inplace=True)

In [ ]:
del titles

### Overview / Plot Synopsis

This is where we need to get into real NLP stuff.

Let's have a peek at our plots:

In [ ]:
[print(_,"\n") for _ in df_boxcoxed_budget.overview.head(10)]

#### Removing stopwords, POS lemmatization

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def process_plot(plot):
    tokens = word_tokenize(plot)
    stops_removed = [w for w in tokens if w not in stop_words]
    punc_removed = [w.lower() for w in stops_removed if w.isalpha()]
    tagged = [(w, get_wordnet_pos(w)) for w in punc_removed]
    return " ".join([lemmatizer.lemmatize(word[0], pos=word[1]) for word in tagged])

In [ ]:
df_boxcoxed_budget['plot_prepped'] = df_boxcoxed_budget.overview.apply(process_plot)

#### Binary Count Vectorizing, TF-IDF Normalizing

From the [sklearn docs](https://scikit-learn.org/stable/modules/feature_extraction.html#):

"The word boundaries-aware variant `char_wb` is especially interesting for languages that use white-spaces for word separation...as it...can increase both the predictive accuracy and convergence speed of classifiers trained using such features while retaining the robustness with regards to misspellings and word derivations."

In [ ]:
bigram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(5, 5),#ngram_range=(1, 2),
                                    token_pattern=r'\b\w+\b', 
                                    min_df=1,
                                    binary=True)

# Plot synopsis texts are not very long: Using binary countvectorizer to help reduce noise

In [ ]:
bigram_vectorizer

In [ ]:
analyze = bigram_vectorizer.build_analyzer()
analyze('Bi-grams are cool!')

In [ ]:
sparse_plots = bigram_vectorizer.fit_transform(df_boxcoxed_budget['plot_prepped'])
sparse_plots

In [ ]:
bigram_columns = bigram_vectorizer.get_feature_names()

In [ ]:
bigram_counts = pd.DataFrame(sparse_plots.toarray(),
                             columns=bigram_columns)

bigram_counts.head()

In [ ]:
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(bigram_counts)

In [ ]:
df_tfidf = pd.DataFrame(tfidf.toarray(), columns=bigram_columns)
df_tfidf.head()

In [ ]:
del bigram_columns
del bigram_counts
del stop_words
del analyze
del df_boxcoxed_budget['overview']
del df_boxcoxed_budget['plot_prepped']
del tfidf
del transformer
del wordnet
del lemmatizer
del languages
del sparse_plots
del bigram_vectorizer
del stopwords

In [ ]:
df_tfidf.columns[:100]

In [ ]:
df_tfidf.columns[5000:5100]

In [ ]:
df_tfidf.shape

Pretty large feature space there. Maybe I'll incorporate some PCA in the modeling pipeline...

In [ ]:
df_boxcoxed_budget.head()

In [ ]:
df_boxcoxed_budget.shape

In [ ]:
df_boxcoxed_budget.columns

In [ ]:
df_boxcoxed_budget.to_pickle('../data/pickles/df_mostly_ready_metadata.pkl')
df_tfidf.to_pickle('../data/pickles/df_tfidf.pkl')

***

# Feature Engineering

## Encoding Everything

In [ ]:
df_mostly_ready_metadata = pd.read_pickle('../data/pickles/df_mostly_ready_metadata.pkl')

In [ ]:
df_mostly_ready_metadata.head(3)

In [ ]:
df_mostly_ready_metadata.dtypes

In [ ]:
lang_matrix = pd.get_dummies(df_mostly_ready_metadata['original_language'],
                             prefix="lang")
lang_matrix.head()

In [ ]:
df_mostly_ready_metadata.drop(['original_language'],axis=1,inplace=True)

In [ ]:
df_mostly_ready_metadata = pd.concat([df_mostly_ready_metadata, 
                                      lang_matrix],axis=1)

In [ ]:
df_mostly_ready_metadata['color'] = df_mostly_ready_metadata['color'].map({"Color":1,
                                                                           "Black and White":0})

df_mostly_ready_metadata.head()

In [ ]:
del df_mostly_ready_metadata['title']

In [ ]:
df_some_binarized = pd.concat([df_mostly_ready_metadata.drop(['fixed_aspect'],axis=1), 
                                pd.get_dummies(df_mostly_ready_metadata.fixed_aspect, prefix="ratio")],axis=1
                              )

In [ ]:
del df_mostly_ready_metadata

In [ ]:
df_some_binarized.head()

In [ ]:
content_ratings = pd.get_dummies(df_some_binarized.content_rating,prefix="content")
content_ratings.head(3)

In [ ]:
df_ratings_binarized = pd.concat([df_some_binarized.drop(['content_rating'],
                                                         axis=1
                                                        ), 
                                  content_ratings],
                                  axis=1
                                )

In [ ]:
del content_ratings
del df_some_binarized

In [ ]:
df_ratings_binarized.head()

In [ ]:
df_ratings_binarized.to_pickle('../data/pickles/df_structured.pkl')

## Separate X and y

In [ ]:
df_structured = pd.read_pickle('../data/pickles/df_structured.pkl')
df_tfidf = pd.read_pickle('../data/pickles/df_tfidf.pkl')

df_tfidf.index = df_structured.index

df_structured.head()

In [ ]:
df_tfidf.head()

In [ ]:
y_colnames = ['revenue', 'imdb_score']

X_structured = df_structured.drop(y_colnames,axis=1)

y_structured = df_structured[y_colnames]

y_structured

In [ ]:
X_structured

## Train / Test Split

Do not fit the scaler on the test set. Do not fit PCA on test set. Do not do feature selection based on anything in the test set.

In [ ]:
X_struct_train, X_struct_test, y_train, y_test = train_test_split(X_structured, 
                                                                  y_structured, 
                                                                  test_size=0.3, 
                                                                  shuffle=True,
                                                                  random_state=42)

I think my approach is going to be to use an ensemble to combine two regressors, one for the structured data and one for the unstructured plot texts

In [ ]:
X_plots_train = df_tfidf.loc[y_train.index]
X_plots_test = df_tfidf.loc[y_test.index]

In [ ]:
del y_structured
del df_structured
del df_tfidf

In [ ]:
X_plots_train.to_pickle('../data/pickles/X_plots_train.pkl')
X_plots_test.to_pickle('../data/pickles/X_plots_test.pkl')
y_train.to_pickle('../data/pickles/y_train.pkl')
y_test.to_pickle('../data/pickles/y_test.pkl')

In [ ]:
del X_plots_train
del X_plots_test

## Checking for Multicollinearity

Are there any redundant features? Are there any features obviously correlated with our dependent var?

In [ ]:
def draw_correlation_heatmap(features):
    # Creating a multi-scatter plot
    corr =  features.corr()
    mask = np.triu(np.ones_like(corr, dtype=np.bool))

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, 
                mask=mask, 
                cmap=cmap,
                vmax=1, 
                center=0,
                square=True, 
                linewidths=.5, 
                cbar_kws={"shrink": .5}).plot()

In [ ]:
draw_correlation_heatmap(X_struct_train)

We have some obviously multicollinear features to lose.

In [ ]:
multicollinear_features = ["shot_only_in_USA", 
                           "shot_in_USA_and_abroad", 
                           "cast_total_facebook_likes_box",
                           "actor_2_facebook_likes_box",
                           "actor_3_facebook_likes_box",
                           "lang_Other",
                           "ratio_1.85",
                           "content_PG-13"]

In [ ]:
X_struct_train.drop(multicollinear_features, axis=1, inplace=True)

# Drop from the test set as well, to maintain same dimensionality
X_struct_test.drop(multicollinear_features, axis=1, inplace=True)

In [ ]:
draw_correlation_heatmap(X_struct_train)

Better! I would use a more mathematically rigorous feature elimination technique, but honestly I'm just planning to try regression models with inherent feature elimination like lasso and random forest.

## Generating Interaction Variables

Now that we've removed the most obviously redundant terms, it might be helpful to generate a number of interaction terms, in case their interplay has some special significance

In [ ]:
poly = PolynomialFeatures(2, 
                          interaction_only=True,
                          include_bias=False)

In [ ]:
poly.fit(X_struct_train)

In [ ]:
interaction_term_names = ["*".join(col.split()) for col in poly.get_feature_names(X_struct_train.columns)]

In [ ]:
X_struct_train_poly = pd.DataFrame(poly.transform(X_struct_train), 
                                   columns = interaction_term_names)

In [ ]:
X_struct_test_poly = pd.DataFrame(poly.transform(X_struct_test), 
                                  columns = interaction_term_names)

In [ ]:
del X_struct_test
del X_struct_train
del X_structured
del interaction_term_names
del multicollinear_features

In [ ]:
X_struct_train_poly.to_pickle('../data/pickles/X_struct_train_poly.pkl')
X_struct_test_poly.to_pickle('../data/pickles/X_struct_test_poly.pkl')

## Scaling Features

Load data:

In [ ]:
X_struct_train_poly = pd.read_pickle('../data/pickles/X_struct_train_poly.pkl')
X_struct_test_poly = pd.read_pickle('../data/pickles/X_struct_test_poly.pkl')
X_plots_train = pd.read_pickle('../data/pickles/X_plots_train.pkl')
X_plots_test = pd.read_pickle('../data/pickles/X_plots_test.pkl')

Make 2 scalers:

In [ ]:
struct_scaler = MinMaxScaler()
plots_scaler = MinMaxScaler()

Fit scaler on training set, apply to both training and testing sets:

In [ ]:
X_struct_train_poly_scaled = pd.DataFrame(struct_scaler.fit_transform(X_struct_train_poly), 
                                         columns = X_struct_train_poly.columns,
                                         index = X_struct_train_poly.index)

X_struct_train_poly_scaled.head()

In [ ]:
# Just transform() test set, not fit_transform()
X_struct_test_poly_scaled = pd.DataFrame(struct_scaler.transform(X_struct_test_poly), 
                                         columns = X_struct_test_poly.columns,
                                         index = X_struct_test_poly.index)

X_struct_test_poly_scaled.head()

In [ ]:
del X_struct_test_poly
del X_struct_train_poly

Also for plot synopsis data:

In [ ]:
X_plots_train_scaled = pd.DataFrame(plots_scaler.fit_transform(X_plots_train), 
                                    columns = X_plots_train.columns,
                                    index = X_plots_train.index)

X_plots_train_scaled

In [ ]:
# Again— only transform(), no fit_transform() here
X_plots_test_scaled = pd.DataFrame(plots_scaler.transform(X_plots_test), 
                                   columns = X_plots_test.columns,
                                   index = X_plots_test.index)

X_plots_test_scaled

In [ ]:
del X_plots_test
del X_plots_train

Pickles:

In [ ]:
X_plots_train_scaled.to_pickle("../data/pickles/X_plots_train_scaled.pkl")
X_plots_test_scaled.to_pickle('../data/pickles/X_plots_test_scaled.pkl')

X_struct_train_poly_scaled.to_pickle("../data/pickles/X_struct_train_poly_scaled.pkl")
X_struct_test_poly_scaled.to_pickle("../data/pickles/X_struct_test_poly_scaled.pkl")

# Model Building / Benchmarking

In [ ]:
X_struct_train_poly_scaled = pd.read_pickle("../data/pickles/X_struct_train_poly_scaled.pkl")
X_struct_test_poly_scaled = pd.read_pickle("../data/pickles/X_struct_test_poly_scaled.pkl")

In [ ]:
y_train = pd.read_pickle('../data/pickles/y_train.pkl')
y_test = pd.read_pickle('../data/pickles/y_test.pkl')

In [ ]:
X_struct_train_poly_scaled.shape[0], X_struct_test_poly_scaled.shape[0]

In [ ]:
y_train.shape[0], y_test.shape[0]

## Baselining

In [ ]:
def get_error_metrics(y_actual, y_preds, name, draw_scatter=True):
    ys = (y_actual, y_preds)
    metrics = dict()
    metrics["r^2"] = r2_score(*ys)
    metrics["MSE"] = mean_squared_error(*ys)
    metrics["Med_AE"] = median_absolute_error(*ys) # Nicely robust to outliers
    rounded_metrics = {k:round(v,3) for k,v in metrics.items()}
    if draw_scatter:
        colors = cm.rainbow(np.linspace(0, 1, len(ys)))
        for y, c in zip(ys, colors):
            plt.scatter(range(len(y)), y, color=c)
    return pd.Series(rounded_metrics, name=name)

### Dummy Regressor

Very very naive model

In [ ]:
dummy_regr_critics = DummyRegressor(strategy="median")
dummy_regr_revenue = DummyRegressor(strategy="mean")

dummy_regr_critics.fit(X_struct_train_poly_scaled, y_train.imdb_score)
dummy_regr_revenue.fit(X_struct_train_poly_scaled, y_train.revenue)

ypred_critics_dummy = pd.Series(dummy_regr_critics.predict(X_struct_test_poly_scaled),
                                index=y_test.index)

del dummy_regr_critics

ypred_revenue_dummy = pd.Series(dummy_regr_revenue.predict(X_struct_test_poly_scaled),
                                index=y_test.index)

del dummy_regr_revenue

In [ ]:
scores_critics_dummy = get_error_metrics(y_test.imdb_score, ypred_critics_dummy, "critics_dummy")
del ypred_critics_dummy
scores_critics_dummy

In [ ]:
scores_revenue_dummy = get_error_metrics(y_test.revenue, ypred_revenue_dummy, "revenue_dummy")
del ypred_revenue_dummy
scores_revenue_dummy

## Basic Models

Let's start with a linear regressor, but one that can prune out seemingly irrelevant features:

### Linear: Lasso Regression

In [ ]:
lasso_regr_critics = LassoCV(cv=5, random_state=0)
lasso_regr_revenue = LassoCV(cv=5, random_state=0)

lasso_regr_critics.fit(X_struct_train_poly_scaled, y_train.imdb_score)
lasso_regr_revenue.fit(X_struct_train_poly_scaled, y_train.revenue)

ypred_critics_lasso = pd.Series(lasso_regr_critics.predict(X_struct_test_poly_scaled),
                                index=y_test.index)

del lasso_regr_critics

ypred_revenue_lasso = pd.Series(lasso_regr_revenue.predict(X_struct_test_poly_scaled),
                                index=y_test.index)

del lasso_regr_revenue

In [ ]:
scores_critics_lasso = get_error_metrics(y_test.imdb_score, 
                                         ypred_critics_lasso, 
                                         "critics_lasso")
del ypred_critics_lasso

scores_critics_lasso

Wow kinda decent at predicting critic scores.

In [ ]:
scores_revenue_lasso = get_error_metrics(y_test.revenue, 
                                         ypred_revenue_lasso, 
                                         "revenue_lasso")

del ypred_revenue_lasso

scores_revenue_lasso

Yikes, significant error on box office scores. Probably has to do with all the zero values in the dependent variable.

People online say that if you have lots of zeroes in your dependent continuous variable, then maybe you should just make a classifier which buckets things into (hopefully) zero and non-zero samples, then train and run your regression model on the non-zero samples. Cool idea, but this is my secondary DV and I'd like to focus on IMdB critic scores for now, so I'll just move on.

**Adding in Plot Synopsis Decomposition**

In [ ]:
X_struct_train_poly_scaled = pd.read_pickle("../data/pickles/X_struct_train_poly_scaled.pkl")
X_struct_test_poly_scaled = pd.read_pickle("../data/pickles/X_struct_test_poly_scaled.pkl")

In [ ]:
X_plots_train_scaled = pd.read_pickle('../data/pickles/X_plots_train_scaled.pkl')
X_plots_test_scaled  = pd.read_pickle('../data/pickles/X_plots_test_scaled.pkl')

In [ ]:
n_text_cols = X_plots_train_scaled.shape[1]
n_text_cols

In [ ]:
X_train = pd.concat([X_struct_train_poly_scaled, 
                     X_plots_train_scaled],
              axis=1)

del X_struct_train_poly_scaled
del X_plots_train_scaled

X_test = pd.concat([X_struct_test_poly_scaled,
                    X_plots_test_scaled],
                   axis=1)


del X_struct_test_poly_scaled
del X_plots_test_scaled

In [ ]:
len(X_train.index)

We will do some PCA on the text features, given the large feature space with the plot text data.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
lasso_critics_withplots = LassoCV(cv=5, random_state=0)

# ypred_critics_withplots = pd.Series(lasso_regr_critics.predict(X_struct_test_poly_scaled),
#                                 index=y_test.index)

In [ ]:
# Define a pipeline to search for the best PCA truncation
pca = PCA()
pipe = Pipeline(steps=[('pca', pca), 
                       ('lasso', lasso_critics_withplots)])

X_digits, y_digits = datasets.load_digits(return_X_y=True)

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'pca__n_components': [5, 15, 30, 45, 64],
    'logistic__C': np.logspace(-4, 4, 4),
}
search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(X_struct_train_poly_scaled, y_train.imdb_score)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

### Non-Parametric: Linear SVR

## Heavyweight Models

### Random Forest

### Neural Net

## Plot Text as Predictive Features

In [ ]:
X_plots_test_scaled = pd.read_pickle('../data/pickles/X_plots_test_scaled.pkl')
X_plots_train_scaled = pd.read_pickle("../data/pickles/X_plots_train_scaled.pkl")

### Dimensionality Reduction